<a href="https://colab.research.google.com/github/ImNotDanish05/2025_KC_TI-1A/blob/main/P11_Convolutional%20Neural%20Network%20(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
print(tf.__version__)

2.18.0


In [2]:
"""
List Dir: https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images
Open File: https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images
"""

base_dir = 'https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images'
train_dir = base_dir + '/train'
validation_dir = base_dir + '/val'

In [6]:
import requests
from urllib.parse import urlparse
def listdirx(dirx):
  response = requests.get(dirx)
  if response.status_code == 200:
      files = response.json()
      print(dirx)
      for file in files:
          print(f"- {file['name']}")
  else:
      print(f"Gagal mengambil data. Status: {response.status_code}")
def list_all_png_from_github_api(api_url, collected=[]):
    collected = []
    response = requests.get(api_url)
    if response.status_code != 200:
        print(f"❌ Gagal mengakses {api_url} | Status: {response.status_code}")
        return collected

    items = response.json()
    for item in items:
        if item['type'] == 'file' and item['name'].endswith('.png'):
            collected.append(item['download_url'])
        elif item['type'] == 'dir':
            list_all_png_from_github_api(item['url'], collected)
    return collected

def download_images(image_urls, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for url in image_urls:
        filename = os.path.basename(urlparse(url).path)
        save_path = os.path.join(save_dir, filename)

        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"✅ Downloaded: {save_dir}/{filename}")
        else:
            print(f"❌ Gagal download: {filename}")

In [7]:
listdirx(train_dir)
listdirx(validation_dir)

https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/train
- .DS_Store
- clean
- messy
https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/val
- clean
- messy


TensorFlow

Documentation: https://keras.io/api/data_loading/image/

In [5]:
train_clean_urls = list_all_png_from_github_api(train_dir + "/clean")
train_messy_urls = list_all_png_from_github_api(train_dir + "/messy")
validation_clean_urls = list_all_png_from_github_api(validation_dir + "/clean")
validation_messy_urls = list_all_png_from_github_api(validation_dir + "/messy")
test_urls = list_all_png_from_github_api(base_dir + "/test")
download_images(train_clean_urls, 'dataset/train/clean')
download_images(train_messy_urls, 'dataset/train/messy')
download_images(validation_clean_urls, 'dataset/val/clean')
download_images(validation_messy_urls, 'dataset/val/messy')
download_images(test_urls, 'dataset/test')
# print(f"{train_dir}/messy")
# for item in validation_clean_urls:
#   print(item)

https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/train/messy
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/0.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/1.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/2.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/3.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/4.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/5.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/6.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/7.png
https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/val/clean/8.png
https://raw.git

In [12]:
print("Train Clean  :", len(os.listdir('/content/dataset/train/clean')))
print("Train Messy  :", len(os.listdir('/content/dataset/train/messy')))
print("Val Clean    :", len(os.listdir('/content/dataset/val/clean')))
print("Val Messy    :", len(os.listdir('/content/dataset/val/messy')))
print("Test         :", len(os.listdir('/content/dataset/test')))
train_dir = '/content/dataset/train'
validation_dir = '/content/dataset/val'

Train Clean  : 96
Train Messy  : 96
Val Clean    : 96
Val Messy    : 96
Test         : 96


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary')

Found 192 images belonging to 2 classes.
Found 192 images belonging to 2 classes.


In [20]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 15, 15, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,529,665 (51.61 MB)

 Trainable params: 13,529,665 (51.61 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# compile model dengan 'adam' optimizer loss function 'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [25]:
# latih model dengan model.fit
model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=20, # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/20
25/25 - 10s - 386ms/step - accuracy: 0.4300 - loss: 0.7057 - val_accuracy: 0.2500 - val_loss: 0.6949
Epoch 2/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


25/25 - 10s - 409ms/step - accuracy: 0.5761 - loss: 0.6919 - val_accuracy: 0.7500 - val_loss: 0.6378
Epoch 3/20
25/25 - 20s - 818ms/step - accuracy: 0.4900 - loss: 0.6978 - val_accuracy: 0.5500 - val_loss: 0.6917
Epoch 4/20
25/25 - 10s - 409ms/step - accuracy: 0.5109 - loss: 0.6934 - val_accuracy: 0.3500 - val_loss: 0.6956
Epoch 5/20
25/25 - 21s - 852ms/step - accuracy: 0.5200 - loss: 0.6929 - val_accuracy: 0.4500 - val_loss: 0.6947
Epoch 6/20
25/25 - 9s - 362ms/step - accuracy: 0.5326 - loss: 0.6924 - val_accuracy: 0.4500 - val_loss: 0.6952
Epoch 7/20
25/25 - 20s - 812ms/step - accuracy: 0.4600 - loss: 0.6947 - val_accuracy: 0.4500 - val_loss: 0.6948
Epoch 8/20
25/25 - 10s - 409ms/step - accuracy: 0.5217 - loss: 0.6928 - val_accuracy: 0.6000 - val_loss: 0.6910
Epoch 9/20
25/25 - 10s - 412ms/step - accuracy: 0.5100 - loss: 0.6938 - val_accuracy: 0.3500 - val_loss: 0.6999
Epoch 10/20
25/25 - 10s - 387ms/step - accuracy: 0.5217 - loss: 0.6929 - val_accuracy: 0.3000 - val_loss: 0.6993
Epo

# **Testing**

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

test_image = "https://i.pinimg.com/736x/31/af/bb/31afbb9580927635c0a076f99bb696fb.jpg"
download_images([test_image], 'dataset/test')

def predict_image_class(model, image_path):
    img = Image.open(image_path).resize((150, 150))
    imgplot = plt.imshow(img)
    x = np.expand_dims(np.array(img), axis=0)
    classes = model.predict(x, batch_size=10)

    if classes[0] == 0:
        print('clean')
    else:
        print('messy')

# Misalkan model adalah objek model yang sudah diinisialisasi sebelumnya

# Mengunggah gambar
image_path = "/content/dataset/test/31afbb9580927635c0a076f99bb696fb.jpg"
predict_image_class(model, image_path)

✅ Downloaded: 31afbb9580927635c0a076f99bb696fb.jpg
